In [0]:
%python
df = spark.read.format("delta").load("abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/bronze/products/")
df.display()

In [0]:
df = df.drop("_ingest_time", "_source_file", "_rescued_data")

In [0]:
from utils.schema_validator import validate_schema
from utils.schema_metadata import silver_table_metadata

table_conf = silver_table_metadata["products"]
source_path = table_conf["source_path"]
target_path = table_conf["target_path"]
schema = table_conf["schema"]
primary_keys = table_conf["primary_keys"]

validate_schema(df,schema)

In [0]:
from utils.scd2 import apply_scd2

apply_scd2(spark, df, target_path, primary_keys)

In [0]:
df2 = spark.read.format("delta").load(target_path)

In [0]:
df2.display()

In [0]:
from utils.data_quality import run_data_quality_checks
dq_metrics=run_data_quality_checks(df, primary_keys)


In [0]:
from utils.audit_logger import log_audit
log_audit(spark, "products", "success", "Loaded with SCD2", dq_metrics.get("row_count", 0))

In [0]:
spark.read.format("delta").load("abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/silver/audit_logs/").display()
